### Example usage for calculate_metrics

In [ ]:
import pickle
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

#from src.main import calculate_metrics

# Copied from src.main as I'm having problems with audio mamba imports
from src.config import CLASSES
from loguru import logger
import sed_eval

def calculate_metrics(pred_event_dict, gt_event_dict):
    logger.warning("Metrics not implemented!")
    # Based on event dictionary

    # Accuracy

    # IoU

    # DCASE SED eval: https://tut-arg.github.io/sed_eval/tutorial.html#id1
    event_based_metrics = sed_eval.sound_event.EventBasedMetrics(CLASSES)
    for file, estimated_event in pred_event_dict.items():
        ref_event = gt_event_dict[file]
        event_based_metrics.evaluate(
            reference_event_list=ref_event,
            estimated_event_list=estimated_event
        )
    print(event_based_metrics)

In [ ]:
# refer to single_stage_yamnet_frame.ipynb (Section "Get test set event predictions") for pickle file generation
pred_event_dict = pickle.load(open('outputs/single_stage_yamnet.pkl', 'rb'))

In [18]:
gt_events = pickle.load(open('data/processed/yamnet/spectrograms_test_list.pkl', 'rb'))
gt_event_dict = {ref_event['file']: [{'file':ref_event['file'], 
                    'event_onset':ref_event['onset'], 
                    'event_offset':ref_event['offset'],
                    'event_label':ref_event['event_label']}]
                    for ref_event in gt_events}

In [19]:
calculate_metrics(pred_event_dict, gt_event_dict)

2025-12-01 01:34:49.543 | WARNING  | __main__:calculate_metrics:14 - Metrics not implemented!


Event based metrics (onset-offset)
  Evaluated length                  : 3340.71 sec
  Evaluated files                   : 500 
  Evaluate onset                    : True 
  Evaluate offset                   : True 
  T collar                          : 200.00 ms
  Offset (length)                   : 50.00 %

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 13.87 %
    Precision                       : 10.92 %
    Recall                          : 19.00 %
  Error rate
    Error rate (ER)                 : 2.34 
    Substitution rate               : 0.02 
    Deletion rate                   : 0.79 
    Insertion rate                  : 1.53 

  Class-wise average metrics (macro-average)
  F-measure
    F-measure (F1)                  : 13.55 %
    Precision                       : 10.70 %
    Recall                          : 18.56 %
  Error rate
    Error rate (ER)                 : 2.36 
    Deletion rate                   : 0.81 
    Insertion rate 